Topic Modelling

In [3]:
import spacy
import re
import spacy_transformers
import torch
import json
nlp = spacy.load('de_dep_news_trf')

In [13]:
with open('BundestagsRedenSample.json') as json_file:
    data = json.load(json_file)

Lowercase is already done during forming dictionary

In [14]:
print(data['ID20100600'][1])

sehr geehrter herr alterspräsident dr. schäuble! also, herr brandner, gleich in der ersten sitzung die braune widerlichkeitsskala in solche höhen zu treiben,das ist immerhin respektvoll.dass ausgerechnet sie willy brandt zitieren, der gegen die nazis gekämpft hat, der niedergekniet ist vor den opfern des aufstandes im warschauer ghetto, dass ausgerechnet sie, die in der tradition der nazis stehen,das hier anbringen, ist abartig, um das in aller klarheit zu sagen. abartig und ekelerregend!liebe kolleginnen und kollegen, sehr geehrte damen und herren von der afd,wenn ich mir die politische lage angucke, dann finde ich zurzeit besonders wichtig: wie entwickelt sich corona? ich finde es wichtig, darüber zu diskutieren, wie wir kinderarmut bekämpfen können. und ich mache mir große sorgen, dass sich hier eine koalition anbahnt, in der das soziale hinten runterfallen wird, weil die stabil marktradikale fdp dafür schon sorgen wird.das ist etwas, was mich und viele menschen bewegt.was ist der a

In [15]:
doc = nlp(data['ID20100600'][1].replace('–', ''))

In [16]:
# doc = nlp(data)

Tokenize

In [29]:
for token in doc:
    if (not token.is_punct 
    and not token.is_currency
    and not token.is_digit
    and not token.is_space
    and not token.is_stop
    and not token.like_num):
        print(token.lemma_)

geehrt
Herr
Alterspräsident
dr
Schäuble
Herr
Brandner
Sitzung
braun
Widerlichkeitsskala
Höhe
treiben
immerhin
respektvoll.dass
ausgerechnet
Willy
Brandt
zitieren
Nazi
kämpfen
niederknieen
Opfer
Aufstand
Warschauer
Ghetto
ausgerechnet
Tradition
Nazi
stehen
anbringen
abartig
Klarheit
sagen
abartig
ekelerregend
lieb
Kollegin
Kollege
geehrt
Dame
Herren
afd
politisch
Lage
angucken
finden
zurzeit
wichtig
entwickeln
Corona
finden
wichtig
diskutieren
Kinderarmut
bekämpfen
machen
Sorge
Koalition
anbahnen
sozial
hinten
runterfallen
stabil
marktradikal
FDP
sorgen
wird.das
Mensch
bewegt.was
afd
wichtig
afd
wichtig
Verunglimpfung
geschlechtergerecht
Sprache
einzig
sinnvoll
Änderungsantrag
sehen
parlamentarisch
verblödet
eigentlich
mal
sagen
Änderungsantrag
einzig
erkennen
offenbar
Bundestag
Gesetzestext
lesen
binnen-i
doppelpunkte
gendersternchen.das
absurd
beantragen
Gesetzestext
völlig
Papierkorb
vorlegen
Fraktion
Vorlage
sozialdemokratisch
Fraktion
unterstützen
halten
Geschäftsordnung
Grundlage


In [30]:
for token in doc:
    print(token.lemma_)

sehr
geehrt
Herr
Alterspräsident
dr
.
Schäuble
--
also
--
Herr
Brandner
--
gleich
in
der
erster
Sitzung
der
braun
Widerlichkeitsskala
in
solcher
Höhe
zu
treiben
--
der
sein
immerhin
respektvoll.dass
ausgerechnet
sie
Willy
Brandt
zitieren
--
der
gegen
der
Nazi
kämpfen
haben
--
der
niederknieen
sein
vor
der
Opfer
der
Aufstand
in
Warschauer
Ghetto
--
dass
ausgerechnet
sie
--
der
in
der
Tradition
der
Nazi
stehen
--
der
hier
anbringen
--
sein
abartig
--
um
der
in
aller
Klarheit
zu
sagen
--
abartig
und
ekelerregend
--
lieb
Kollegin
und
Kollege
--
sehr
geehrt
Dame
und
Herren
von
der
afd
--
wenn
ich
sich
der
politisch
Lage
angucken
--
dann
finden
ich
zurzeit
besonders
wichtig
--
wie
entwickeln
sich
Corona
--
ich
finden
es
wichtig
--
darüber
zu
diskutieren
--
wie
wir
Kinderarmut
bekämpfen
können
--
und
ich
machen
mir
groß
Sorge
--
dass
sich
hier
ein
Koalition
anbahnen
--
in
der
der
sozial
hinten
runterfallen
werden
--
weil
der
stabil
marktradikal
FDP
dafür
schon
sorgen
wird.das
sein
etwas
--
we